In [201]:
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream
import twitter
import os
import urllib
import json
import datetime as dt
from dateutil import parser
from time import sleep
import re

# twitter tokens, keys, secrets, and Twitter handle in the following variables
CONSUMER_KEY = 'E19oBd9qdE1wXWiyixMfrubbI'
CONSUMER_SECRET ='IU5qiEwHJgKAVJN0fXMux79yIzsMISSjLORB3j8sXXvUFddlnV'
OAUTH_TOKEN = '2749655899-u4geaWEZHlCXtvk12wlVJ84JmSX4HIQuD3FEsDQ'
OAUTH_TOKEN_SECRET = 'NlUL020uY5mXW4nFonFI2PgWDMguv6V2aF9QGGQkCAly8'
TWITTER_HANDLE = "ulfaslak"
TWITTER_ID = 2749655899

def search_tweets(q, count=100, result_type="recent", lang='en'):
    return t.search.tweets(q=q, result_type=result_type, count=count, lang=lang)

def get_user_links_likes(user):
    return [
        (user, tweet['user']['screen_name'], str(parser.parse(tweet['created_at'])))
        for tweet in t.favorites.list(screen_name=user, count=100)
        if tweet['user']['screen_name'] in users
    ]

def update_users(new_users):
    with open('users', 'r') as fp:
        users = json.load(fp)
    with open('users', 'w') as fp:
        users = sorted(set(users) | set(new_users))
        json.dump(users, fp)
    return users
    
def update_links(new_links, filename):
    with open(filename, 'r') as fp:
        links = [tuple(l.split(",")) for l in fp.read().split("\n")[1:]]
    with open(filename, 'w') as fp:
        links = sorted(set(links) | set(new_links))
        fp.write("source,target,datetime\n")
        fp.write("\n".join([",".join(l) for l in links]))
        
t = Twitter(auth=OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET))

# Get users that tweeted with #netsci2018 hashtag
print "Searching for tweets with hashtag '#NetSci2018'"
collection_tweets = search_tweets("#netsci2018")['statuses']
print "Loaded %d tweets"  % len(collection_tweets),

print "from",
users = sorted(set(update_users([tweet['user']['screen_name'] for tweet in collection_tweets])))  # Everybody who has tweeted
#users = users | set([
#    tagged_user
#    for tweet in collection_tweets
#    for tagged_user in re.findall(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9_]+)", tweet['text'])
#])
print "%d different users:" %len(users)

print "\n".join(users)
print "\n... saving users"

# For each, produce links from their favorites
print "\nGetting like-links for each user:"
links_likes = []
for user in users:
    try:
        links_user = get_user_links_likes(user)
    except twitter.TwitterHTTPError:
        print "Warning: Rate limit exceeded (user: %s), waiting 15 minutes" % user
        sleep(60 * 15)
        links_user = get_user_links_likes(user)
        continue
        
    print user, len(links_user)
    links_likes.extend(links_user)

print "\nTotal:", len(links)

# Produce retweet links
links_retweets = []
for tweet in collection_tweets:
    if 'retweeted_status' in tweet and tweet['retweeted_status']['user']['screen_name'] in users:
        links_retweets.append(
            (tweet['user']['screen_name'], tweet['retweeted_status']['user']['screen_name'], str(parser.parse(tweet['created_at'])))
        )
        
# Produce tag links
links_tags = []
for tweet in collection_tweets:
    if 'retweeted_status' not in tweet:
        for tagged_user in re.findall(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9_]+)", tweet['text']):
            if tagged_user:
                links_tags.append(
                    (tweet['user']['screen_name'], tagged_user, str(parser.parse(tweet['created_at'])))
                )
                    


print "\n... saving links"
update_links(links_likes, "links_likes.csv")
update_links(links_retweets, "links_retweets.csv")
update_links(links_tags, "links_tags.csv")

Searching for tweets with hashtag '#NetSci2018'
Loaded 100 tweets from 107 different users:
AntoViral
BallandiesMC
CGraciaLazaro
DAPSworkshop
FerraginaTeach
FrancescoBonchi
Ghoshal_G
Giuseppe_Jurman
HorEyeJen
ISI_Fondazione
IacopoIacopini
JacobBiamonte
JeroenJGGeurts
Knhannah
KoljaKleineberg
LHDnets
LeonieMueck
LindaDouw
Machine_Leaning
MangioniG
MilleNeige
MindScienceFdn
NetContagions
NetSciX
PLOS
PLOSNTDs
PLOSONE
ParagonSci_Inc
PiratePeel
RachaelMilwid
RenaudLambiotte
SamuelJenness
Sandro_Meloni
Schr0dingersKat
SebinoStram
SoBigData
SophieMeakin
Tatmann9
_AlexArenas
_jgyou
a_baronca
aldoceccarelli
all_are
anduviera
ayirpelle
baldellino
bansallab
bigdatatales
ciro
cosnet_bifi
dan_marinazzo
deaneckles
diestra77
droneale
dubitareaude
elisa_omodei
emanudelfava
eugeValdano
ewancolman
fede7j
figshare
jabawack
jefferywang0429
joergheber
juniperlov
k_vanderwaal
kristamenglish
laetitiagvn
leoferres
leriomaggio
lucpappalard
luyibov
manlius84
mar7k
marco_java
marianobegue
mario_paolucci
martika

In [200]:
for i, user in enumerate(users):
    if i < 75: continue
    try:
        links_user = get_user_links_likes(user)
    except twitter.TwitterHTTPError:
        print "Warning: Rate limit exceeded (user: %s), waiting 15 minutes" % user
        sleep(60 * 15)
        links_user = get_user_links_likes(user)
        continue
        
    print user, len(links_user)
    links_likes.extend(links_user)

print "\nTotal like links:", len(links_likes)

# Produce retweet links
links_retweets = []
for tweet in collection_tweets:
    if 'retweeted_status' in tweet and tweet['retweeted_status']['user']['screen_name'] in users:
        links_retweets.append(
            (tweet['user']['screen_name'], tweet['retweeted_status']['user']['screen_name'], str(parser.parse(tweet['created_at'])))
        )
        
print "\nTotal retweet links:", len(links_likes)
        
# Produce tag links
links_tags = []
for tweet in collection_tweets:
    if 'retweeted_status' not in tweet:
        for tagged_user in re.findall(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9_]+)", tweet['text']):
            if tagged_user:
                links_tags.append(
                    (tweet['user']['screen_name'], tagged_user, str(parser.parse(tweet['created_at'])))
                )
                
print "\nTotal tag links:", len(links_likes)
                    


print "\n... saving links"
update_links(links_likes, "links_likes.csv")
update_links(links_retweets, "links_retweets.csv")
update_links(links_tags, "links_tags.csv")

JeroenJGGeurts 1
dubitareaude 0
SamuelJenness 4
kristamenglish 1
netsci2018 33
sdwfrost 3
niamhoconnor73 1
ryanjgallag 14
bansallab 11
IacopoIacopini 8
tinaeliassi 14
baldellino 3
Ghoshal_G 5
SoBigData 2
ayirpelle 0
k_vanderwaal 0
quant_success 5
deaneckles 2
noamross 2
sycramore 0
net_science 17
ISI_Fondazione 10
lucpappalard 6
joergheber 9
SophieMeakin 4

Total: 27

... saving links


In [194]:
update_links(links_retweets, "links_retweets.csv")
update_links(links_tags, "links_tags.csv")

[]
[]


In [199]:
users

{u'AntoViral',
 u'BallandiesMC',
 u'CGraciaLazaro',
 u'DAPSworkshop',
 u'FrancescoBonchi',
 u'Ghoshal_G',
 u'Giuseppe_Jurman',
 u'HorEyeJen',
 u'ISI_Fondazione',
 u'IacopoIacopini',
 u'JacobBiamonte',
 u'JeroenJGGeurts',
 u'KoljaKleineberg',
 u'LHDnets',
 u'LeonieMueck',
 u'LindaDouw',
 u'Machine_Leaning',
 u'MangioniG',
 u'MilleNeige',
 u'MindScienceFdn',
 u'NetContagions',
 u'NetSciX',
 u'PLOS',
 u'PLOSNTDs',
 u'PLOSONE',
 u'ParagonSci_Inc',
 u'PiratePeel',
 u'RachaelMilwid',
 u'RenaudLambiotte',
 u'SamuelJenness',
 u'Sandro_Meloni',
 u'Schr0dingersKat',
 u'SoBigData',
 u'SophieMeakin',
 u'Tatmann9',
 u'_AlexArenas',
 u'_jgyou',
 u'a_baronca',
 u'all_are',
 u'anduviera',
 u'ayirpelle',
 u'baldellino',
 u'bansallab',
 u'bigdatatales',
 u'ciro',
 u'cosnet_bifi',
 u'dan_marinazzo',
 u'deaneckles',
 u'diestra77',
 u'dubitareaude',
 u'elisa_omodei',
 u'emanudelfava',
 u'ewancolman',
 u'fede7j',
 u'figshare',
 u'jabawack',
 u'jefferywang0429',
 u'joergheber',
 u'juniperlov',
 u'k_vanderwaa

In [169]:
links_retweets

In [172]:
tweet['retweeted_status']['user']['screen_name']

u'cosnet_bifi'

In [181]:
[name for tweet in collection_tweets for name in re.findall(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9_]+)", tweet['text'])]

[u'cosnet_bifi',
 u'serg_arregui',
 u'Sanz_Jq',
 u'IacopoIacopini',
 u'netplexity',
 u'cosnet_bifi',
 u'serg_arregui',
 u'Sanz_Jq',
 u'emanudelfava',
 u'jabawack',
 u'oneofyen',
 u'aaronclauset',
 u'leman_akogl',
 u'emanudelfava',
 u'jabawack',
 u'ciro',
 u'jabawack',
 u'TAlexPerkins',
 u'casey_zipfel',
 u'mtizzoni',
 u'EcoHealthNYC',
 u'cosnet_bifi',
 u'netsci2018',
 u'PiratePeel',
 u'serg_arregui',
 u'Sanz_Jq',
 u'lucpappalard',
 u'fede7j',
 u'netsci2018',
 u'lucpappalard',
 u'fede7j',
 u'netsci2018',
 u'rionbr',
 u'LuisMateusRocha',
 u'CNS',
 u'sdwfrost',
 u'chpoletto',
 u'netsci2018',
 u'mtizzoni',
 u'cosnet_bifi',
 u'eugeValdano',
 u'shaipilo',
 u'Giuseppe_Jurman',
 u'PLOSONE',
 u'NetSci2018',
 u'LeonieMueck',
 u'PLOSONE',
 u'NetSci2018',
 u'LeonieMueck',
 u'sdwfrost',
 u'm_rosvall',
 u'tiagopeixoto',
 u'shaipilo',
 u'Giuseppe_Jurman',
 u'mtizzoni',
 u'LHDnets',
 u'PiratePeel',
 u'victorveitch',
 u'aaronclauset',
 u'Giuseppe_Jurman',
 u'a_baronca',
 u'a_baronca',
 u'PLOSONE',
 u'N